In [50]:
# Question 1 : Generate data from assignment1 code.py. 

import numpy as np
import pickle
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D))
y = np.zeros(N*K, dtype='uint8')
for j in range(K):
  ix = range(N*j,N*(j+1))
  r = np.linspace(0.0,1,N) # radius
  t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
  X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
  y[ix] = j

pickle.dump(X,open('dataX.pickle','wb'))
pickle.dump(y,open('dataY.pickle','wb'))


In [51]:
#Imports 
import numpy as np #Represent ndarrays a.k.a. tensors
import matplotlib.pyplot as plt #For plotting
np.random.seed(0) #For repeatability of the experiment
import pickle #To read data for this experiment

#Read data
X = pickle.load(open('dataX.pickle','rb'))
y = pickle.load(open('dataY.pickle','rb'))


#Define some local varaibles
D = X.shape[1] #Number of features
K = max(y)+1 #Number of classes assuming class index starts from 0

In [52]:
# Question 2 : Split the data into test and train (20%:80%)

test_size = 0.2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

trainX, testX, trainY, testY = train_test_split(X,y, test_size = test_size, stratify = y)

In [53]:
# Start with an initialize parameters randomly

W = 0.01 * np.random.randn(D,K) #random weights
#zero bias terms
b = np.zeros((1,K))

#Initialize hyper-parameter value (not optimized)
reg = 1e-3 # regularization strength

In [54]:
# Question 3 : Build a linear classifier assuming the multiclass logistic loss and an `2 regularization for the weights only. 
#              Report the prediction accuracy on the training data and the test data and show appropriate plots.

#Perform batch SGD using backprop

#For simplicity we will take the batch size to be the same as number of examples
num_examples = trainX.shape[0]

#Initial value for the Gradient Descent Parameter
step_size = 1e-0 #Also called learning rate

num_iterations = 200 #200 iterations

# gradient descent loop
for i in range(num_iterations): 
  
    # evaluate class scores, [N x K]
    # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
    scores = np.dot(trainX, W) + b 
    
    # compute the class probabilities
    exp_scores = np.exp(scores)
    
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

    #Compute loss using snippet of LogLoss fucntion created in Q3
    #convert the y label to a N * K array 
    actual = np.zeros(probs.shape)
    rows = actual.shape[0]
    
    actual[np.arange(rows), trainY.astype(int)] = 1
    
    #compute data loss
    data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

    # L2 regularization term
    L2 = np.sum(W ** 2) #not multiplied by 0.5

    # the total loss
    loss = data_loss + reg * L2

    #if i % 10 == 0:
        #print "iteration %d: loss %f" % (i, loss)
        #print "iteration %d: loss %f" % (i, loss_old)
  
    # compute the gradient on scores
    dscores = probs
    dscores[range(num_examples),trainY] -= 1
    dscores /= num_examples
  
    # backpropate the gradient to the parameters (W,b)
    dW = np.dot(trainX.T, dscores)
    db = np.sum(dscores, axis=0, keepdims=True)
  
    dW += reg*W # regularization gradient
  
    # perform a parameter update
    W += -step_size * dW
    b += -step_size * db


scores = np.dot(testX, W) + b
predicted_class = np.argmax(scores, axis=1)

print ('test accuracy: %.2f' % (np.mean(predicted_class == testY)))

test accuracy: 0.53


In [26]:
# Question 4 : Introduce a cross validation scheme and justify your choice of parameters. What is the validation accuracy 
#              compare to the test accuracy

#Using k-fold validation
from sklearn.model_selection import KFold

#Initial value for the Gradient Descent Parameter -- to be held constant
step_size = 1e-0 #Also called learning rate
num_iterations = 200 #200 iterations

#create k-fold criteria
num_folds = 3
k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

#initialize accuraices (for each fold) to empty list
accuracies = list()

k = 1 #for printing

for train_indices, test_indices in k_fold.split(X):

    # Start with an initialize parameters randomly, for each fold
    #random weights
    W = 0.01 * np.random.randn(D,K)
    #zero bias terms
    b = np.zeros((1,K))
    
    # We use 'list' to copy, in order to 'pop' later on
    X_train_k = X[train_indices,]
    X_test_k  = X[test_indices,]
    
    y_train_k = y[train_indices]
    y_test_k  = y[test_indices]
    
    #For simplicity we will take the batch size to be the same as number of examples
    num_examples = X_train_k.shape[0]

    # gradient descent loop
    for i in range(num_iterations): 
        
        # evaluate class scores, [N x K]
        # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
        scores = np.dot(X_train_k, W) + b 

        # compute the class probabilities
        exp_scores = np.exp(scores)

        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

        #Compute loss using snippet of LogLoss fucntion created in Q3
        #convert the y label to a N * K array 
        actual = np.zeros(probs.shape)
        rows = actual.shape[0]

        actual[np.arange(rows), y_train_k.astype(int)] = 1

        #compute data loss
        data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

        # L2 regularization term
        L2 = np.sum(W ** 2) #not multiplied by 0.5

        # the total loss
        loss = data_loss + reg * L2

        # compute the gradient on scores
        dscores = probs
        dscores[range(num_examples),y_train_k] -= 1
        dscores /= num_examples

        # backpropate the gradient to the parameters (W,b)
        dW = np.dot(X_train_k.T, dscores)
        db = np.sum(dscores, axis=0, keepdims=True)

        dW += reg*W # regularization gradient

        # perform a parameter update
        W += -step_size * dW
        b += -step_size * db
    
    #print sum(X_test)
    #print y_train_k
    
    #compute scores for each fold
    scores = np.dot(X_test_k, W) + b
    predicted_class = np.argmax(scores, axis=1)
    #print predicted_class
    #accuracy for fold
    fold_accuracy = np.mean(predicted_class == y_test_k)
    
    print ('Validation Accuracy for fold %d: %.2f' % (k, fold_accuracy))
    accuracies.append(fold_accuracy)
    
    k += 1


print("Accuracy: %0.2f (+/- %0.2f)" % (np.mean(accuracies), np.std(accuracies) * 2))

Validation Accuracy for fold 1: 0.47
Validation Accuracy for fold 2: 0.50
Validation Accuracy for fold 3: 0.58
Accuracy: 0.52 (+/- 0.09)


In [55]:
# Question 5 : What is the sensitivity of the model’s performance to different learning rates and the number of gradient descent
#              iterations. Describe via suitable plots.

#Defining expand grid function 
import itertools
import pandas as pd

def expand_grid(data_dict):
    rows = itertools.product(*data_dict.values())
    return pd.DataFrame.from_records(rows, columns=data_dict.keys())


In [56]:
# Creating training grid
trainGrid = expand_grid({ 'num_iters':[10,25,50,75,100,150, 200, 250, 300, 400,500,750,1000],
                          'step_size': [1e-3,1e-2,1e-1,5e-1,1,1.5],
                         
                           #create a column to store accuracies in
                          'accuracy_mean': [0.0],
                         
                           #create a column to store accuract std's in
                          'accuracy_std' : [0.0] 
                        })

In [57]:
# Using the training Grid and k-fold cross validation to train models

from sklearn.model_selection import KFold

num_folds = 3

#iterate over traning grid
for index, row in trainGrid.iterrows():

    #initialize accuraices (for each fold) to empty list
    accuracies = list()

    #Initial value for the Gradient Descent Parameter
    step_size = row['step_size'] #Also called learning rate
    num_iterations = int(row['num_iters']) # iterations

    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    k = 1 #for printing
    
    #for each row of training grid (which specifies paramaters for training), perform k-fold cv to train
    for train_indices, test_indices in k_fold.split(X):

        # Start with an initialize parameters randomly, for each fold
        #random weights
        W = 0.01 * np.random.randn(D,K)
        #zero bias terms
        b = np.zeros((1,K))
        
        #training and testing for each iteration of k-fold cv
        X_train_k = X[train_indices,]
        X_test_k  = X[test_indices,]

        y_train_k = y[train_indices]
        y_test_k  = y[test_indices]

        #For simplicity we will take the batch size to be the same as number of examples
        num_examples = X_train_k.shape[0]

        # gradient descent loop
        for i in range(num_iterations): 

            # evaluate class scores, [N x K]
            # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
            scores = np.dot(X_train_k, W) + b 

            # compute the class probabilities
            exp_scores = np.exp(scores)


            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

            #Compute loss using snippet of LogLoss fucntion created in Q3
            #convert the y label to a N * K array 
            actual = np.zeros(probs.shape)
            rows = actual.shape[0]

            actual[np.arange(rows), y_train_k.astype(int)] = 1

            #compute data loss
            data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

            # L2 regularization term
            L2 = np.sum(W ** 2) #not multiplied by 0.5

            # the total loss
            loss = data_loss + reg * L2

            # compute the gradient on scores
            dscores = probs
            dscores[range(num_examples),y_train_k] -= 1
            dscores /= num_examples

            # backpropate the gradient to the parameters (W,b)
            dW = np.dot(X_train_k.T, dscores)
            db = np.sum(dscores, axis=0, keepdims=True)

            dW += reg*W # regularization gradient

            # perform a parameter update
            W += -step_size * dW
            b += -step_size * db


        #compute scores for each fold
        scores = np.dot(X_test_k, W) + b
        predicted_class = np.argmax(scores, axis=1)
        #accuracy for fold
        fold_accuracy = np.mean(predicted_class == y_test_k)

        #print 'validation accuracy for fold %d: %.2f' % (k, fold_accuracy)
        accuracies.append(fold_accuracy)

        k += 1 #only for printing purposes
    
    #set mean accuracy and accuracy std in training grid
    trainGrid.set_value(index,'accuracy_mean', np.mean(accuracies))
    trainGrid.set_value(index,'accuracy_std', np.std(accuracies) * 2)

#Printing the accuracies and std deviations now
print (trainGrid) 


    num_iters  step_size  accuracy_mean  accuracy_std
0          10      0.001       0.310000      0.065320
1          10      0.010       0.420000      0.114310
2          10      0.100       0.563333      0.067987
3          10      0.500       0.520000      0.099331
4          10      1.000       0.510000      0.086410
5          10      1.500       0.506667      0.082192
6          25      0.001       0.290000      0.142361
7          25      0.010       0.506667      0.033993
8          25      0.100       0.533333      0.123648
9          25      0.500       0.506667      0.082192
10         25      1.000       0.506667      0.065997
11         25      1.500       0.513333      0.067987
12         50      0.001       0.463333      0.312552
13         50      0.010       0.536667      0.065997
14         50      0.100       0.526667      0.118134
15         50      0.500       0.506667      0.065997
16         50      1.000       0.513333      0.083799
17         50      1.500    

In [58]:
# Writing results to csv for plotting
# trainGrid.to_csv('assignment1_4_5.csv')

topResults = trainGrid.sort_values('accuracy_mean', ascending=0)

print (topResults.head(15))

print (topResults['step_size'][0])

    num_iters  step_size  accuracy_mean  accuracy_std
54        400      0.001       0.566667      0.108730
2          10      0.100       0.563333      0.067987
66        750      0.001       0.560000      0.071181
25        100      0.010       0.556667      0.108730
72       1000      0.001       0.556667      0.077172
48        300      0.001       0.553333      0.052493
19         75      0.010       0.553333      0.067987
60        500      0.001       0.550000      0.099331
31        150      0.010       0.546667      0.120370
43        250      0.010       0.536667      0.118134
13         50      0.010       0.536667      0.065997
37        200      0.010       0.533333      0.094281
8          25      0.100       0.533333      0.123648
36        200      0.001       0.530000      0.086410
49        300      0.010       0.530000      0.114310
0.001


In [59]:
# Question 6 : What is the sensitivity of the model’s performance to different regularization parameter values. Find the best 
#              regularization parameter using an exhaustive search procedure. Describe your choice via suitable plots. What is 
#              the performance difference between using regularization and no regularization?

#Using the best fit found in the last part, we'll try to find the best regularization parameter:

trainGrid_reg = expand_grid({ 'reg': np.arange(0, 1, 0.01),
                              'num_iters':[200],
                              'step_size': [1e-0],
                              'accuracy_mean': [0.0], #create a column to store accuracies in
                              'accuracy_std' : [0.0] #create a column to store accuract std's in
                        })

In [60]:
# Using the training Grid and k-fold cross validation to train models

from sklearn.model_selection import KFold

num_folds = 3

#iterate over traning grid
for index, row in trainGrid_reg.iterrows():
    
    #set reg other than global
    reg = row['reg']
    
    #initialize accuraices (for each fold) to empty list
    accuracies = list()

    #Initial value for the Gradient Descent Parameter
    step_size = row['step_size'] #Also called learning rate
    num_iterations = int(row['num_iters']) #200 iterations

    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    k = 1 #for printing
    
    #for each row of training grid (which specifies paramaters for training), perform k-fold cv to train
    for train_indices, test_indices in k_fold.split(X):

        # Start with an initialize parameters randomly, for each fold
        #random weights
        W = 0.01 * np.random.randn(D,K)
        #zero bias terms
        b = np.zeros((1,K))
        
        #training and testing for each iteration of k-fold cv
        X_train_k = X[train_indices,]
        X_test_k  = X[test_indices,]

        y_train_k = y[train_indices]
        y_test_k  = y[test_indices]

        #For simplicity we will take the batch size to be the same as number of examples
        num_examples = X_train_k.shape[0]

        # gradient descent loop
        for i in range(num_iterations): 

            # evaluate class scores, [N x K]
            # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
            scores = np.dot(X_train_k, W) + b 

            # compute the class probabilities
            exp_scores = np.exp(scores)


            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

            #Compute loss using snippet of LogLoss fucntion created in Q3
            #convert the y label to a N * K array 
            actual = np.zeros(probs.shape)
            rows = actual.shape[0]

            actual[np.arange(rows), y_train_k.astype(int)] = 1

            #compute data loss
            data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

            # L2 regularization term
            L2 = np.sum(W ** 2) #not multiplied by 0.5

            # the total loss
            loss = data_loss + reg * L2

            # compute the gradient on scores
            dscores = probs
            dscores[range(num_examples),y_train_k] -= 1
            dscores /= num_examples

            # backpropate the gradient to the parameters (W,b)
            dW = np.dot(X_train_k.T, dscores)
            db = np.sum(dscores, axis=0, keepdims=True)

            dW += reg*W # regularization gradient

            # perform a parameter update
            W += -step_size * dW
            
#use the training Grid and k-fold cross validation to train models

from sklearn.model_selection import KFold

num_folds = 3

#iterate over traning grid
for index, row in trainGrid_reg.iterrows():
    
    #set reg other than global
    reg = row['reg']
    
    #initialize accuraices (for each fold) to empty list
    accuracies = list()

    #Initial value for the Gradient Descent Parameter
    step_size = row['step_size'] #Also called learning rate
    num_iterations = int(row['num_iters']) #200 iterations

    k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=0)

    k = 1 #for printing
    
    #for each row of training grid (which specifies paramaters for training), perform k-fold cv to train
    for train_indices, test_indices in k_fold.split(X):

        # Start with an initialize parameters randomly, for each fold
        #random weights
        W = 0.01 * np.random.randn(D,K)
        #zero bias terms
        b = np.zeros((1,K))
        
        #training and testing for each iteration of k-fold cv
        X_train_k = X[train_indices,]
        X_test_k  = X[test_indices,]

        y_train_k = y[train_indices]
        y_test_k  = y[test_indices]

        #For simplicity we will take the batch size to be the same as number of examples
        num_examples = X_train_k.shape[0]

        # gradient descent loop
        for i in range(num_iterations): 

            # evaluate class scores, [N x K]
            # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
            scores = np.dot(X_train_k, W) + b 

            # compute the class probabilities
            exp_scores = np.exp(scores)


            probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

            #Compute loss using snippet of LogLoss fucntion created in Q3
            #convert the y label to a N * K array 
            actual = np.zeros(probs.shape)
            rows = actual.shape[0]

            actual[np.arange(rows), y_train_k.astype(int)] = 1

            #compute data loss
            data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

            # L2 regularization term
            L2 = np.sum(W ** 2) #not multiplied by 0.5

            # the total loss
            loss = data_loss + reg * L2

            # compute the gradient on scores
            dscores = probs
            dscores[range(num_examples),y_train_k] -= 1
            dscores /= num_examples

            # backpropate the gradient to the parameters (W,b)
            dW = np.dot(X_train_k.T, dscores)
            db = np.sum(dscores, axis=0, keepdims=True)

            dW += reg*W # regularization gradient

            # perform a parameter update
            W += -step_size * dW
            b += -step_size * db


        #compute scores for each fold
        scores = np.dot(X_test_k, W) + b
        predicted_class = np.argmax(scores, axis=1)
        #accuracy for fold
        fold_accuracy = np.mean(predicted_class == y_test_k)

        #print 'validation accuracy for fold %d: %.2f' % (k, fold_accuracy)
        accuracies.append(fold_accuracy)

        k += 1 #only for printing purposes
    
    #set mean accuracy and accuracy std in training grid
    trainGrid_reg.set_value(index,'accuracy_mean', np.mean(accuracies))
    trainGrid_reg.set_value(index,'accuracy_std', np.std(accuracies) * 2)
    b += -step_size * db


    #compute scores for each fold
    scores = np.dot(X_test_k, W) + b
    predicted_class = np.argmax(scores, axis=1)
    #accuracy for fold
    fold_accuracy = np.mean(predicted_class == y_test_k)

    #print 'validation accuracy for fold %d: %.2f' % (k, fold_accuracy)
    accuracies.append(fold_accuracy)

    k += 1 #only for printing purposes
    
    #set mean accuracy and accuracy std in training grid
    trainGrid_reg.set_value(index,'accuracy_mean', np.mean(accuracies))
    trainGrid_reg.set_value(index,'accuracy_std', np.std(accuracies) * 2)
    
print (trainGrid_reg)

     reg  num_iters  step_size  accuracy_mean  accuracy_std
0   0.00        200        1.0         0.5400      0.100995
1   0.01        200        1.0         0.5275      0.087607
2   0.02        200        1.0         0.5300      0.081240
3   0.03        200        1.0         0.5300      0.081240
4   0.04        200        1.0         0.5250      0.094340
5   0.05        200        1.0         0.5300      0.103923
6   0.06        200        1.0         0.5275      0.107121
7   0.07        200        1.0         0.5300      0.100995
8   0.08        200        1.0         0.5300      0.100995
9   0.09        200        1.0         0.5325      0.095263
10  0.10        200        1.0         0.5325      0.095263
11  0.11        200        1.0         0.5375      0.105238
12  0.12        200        1.0         0.5425      0.115217
13  0.13        200        1.0         0.5375      0.105238
14  0.14        200        1.0         0.5375      0.105238
15  0.15        200        1.0         0

In [61]:
#write results to csv for plotting
#trainGrid_reg.to_csv('assignment1_4_6.csv')

topResults_reg = trainGrid_reg.sort_values('accuracy_mean', ascending=0)

print (topResults_reg.head(15))

     reg  num_iters  step_size  accuracy_mean  accuracy_std
52  0.52        200        1.0         0.6325      0.065383
53  0.53        200        1.0         0.6325      0.065383
54  0.54        200        1.0         0.6325      0.065383
51  0.51        200        1.0         0.6300      0.066332
55  0.55        200        1.0         0.6300      0.066332
50  0.50        200        1.0         0.6250      0.075498
48  0.48        200        1.0         0.6250      0.091104
56  0.56        200        1.0         0.6250      0.057446
49  0.49        200        1.0         0.6225      0.077942
60  0.60        200        1.0         0.6225      0.049749
61  0.61        200        1.0         0.6225      0.049749
58  0.58        200        1.0         0.6200      0.048990
57  0.57        200        1.0         0.6200      0.048990
62  0.62        200        1.0         0.6200      0.058310
63  0.63        200        1.0         0.6200      0.058310


In [62]:
# Question 7 : What is the sensitivity of the model’s performance with respect to a different test train split (50%:50%).

test_size = 0.2

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

#trainX, testX, trainY, testY = train_test_split(X,y, test_size = test_size, stratify = y)

trainGrid_diffSplits = expand_grid({ 'test_size' : np.arange(0.05, 0.55, 0.05),
                                     'accuracy': [0.0]
                          #'num_iters':[10,25,50,75,100,150, 200, 250, 300, 400,500,750,1000],
                          #'step_size': [1e-3,1e-2,1e-1,5e-1,1,1.5],
                          #'accuracy_mean': [0.0], #create a column to store accuracies in
                          #'accuracy_std' : [0.0] #create a column to store accuract std's in
                        })
print (trainGrid_diffSplits)

   test_size  accuracy
0       0.05       0.0
1       0.10       0.0
2       0.15       0.0
3       0.20       0.0
4       0.25       0.0
5       0.30       0.0
6       0.35       0.0
7       0.40       0.0
8       0.45       0.0
9       0.50       0.0


In [63]:
#Initial value for the Gradient Descent Parameter
step_size = 1e-0 #Also called learning rate

num_iterations = 200 #200 iterations

#Initialize hyper-parameter value (not optimized)
reg = 1e-3 # regularization strength

#iterate over traning grid
for index, row in trainGrid_diffSplits.iterrows():
    #random weights
    W = 0.01 * np.random.randn(D,K)
    #zero bias terms
    b = np.zeros((1,K))

    trainX, testX, trainY, testY = train_test_split(X,y, test_size = row['test_size'], stratify = y)
    
    num_examples = trainX.shape[0]
    
    # gradient descent loop
    for i in range(num_iterations): 

        # evaluate class scores, [N x K]
        # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
        scores = np.dot(trainX, W) + b 

        # compute the class probabilities
        exp_scores = np.exp(scores)

        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

        #Compute loss using snippet of LogLoss fucntion created in Q3
        #convert the y label to a N * K array 
        actual = np.zeros(probs.shape)
        rows = actual.shape[0]

        actual[np.arange(rows), trainY.astype(int)] = 1

        #compute data loss
        data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

        # L2 regularization term
        L2 = np.sum(W ** 2) #not multiplied by 0.5

        # the total loss
        loss = data_loss + reg * L2

        #if i % 10 == 0:
            #print "iteration %d: loss %f" % (i, loss)
            #print "iteration %d: loss %f" % (i, loss_old)

        # compute the gradient on scores
        dscores = probs
        dscores[range(num_examples),trainY] -= 1
        dscores /= num_examples

        # backpropate the gradient to the parameters (W,b)
        dW = np.dot(trainX.T, dscores)
        db = np.sum(dscores, axis=0, keepdims=True)

        dW += reg*W # regularization gradient

        # perform a parameter update
        W += -step_size * dW
        b += -step_size * db

    scores = np.dot(testX, W) + b
    predicted_class = np.argmax(scores, axis=1)
    trainGrid_diffSplits.set_value(index,'accuracy', np.mean(predicted_class == testY))
    print ('test accuracy with test split of %.2f: %.2f' % (row['test_size'], np.mean(predicted_class == testY)))


test accuracy with test split of 0.05: 0.60
test accuracy with test split of 0.10: 0.53
test accuracy with test split of 0.15: 0.54
test accuracy with test split of 0.20: 0.45
test accuracy with test split of 0.25: 0.51
test accuracy with test split of 0.30: 0.50
test accuracy with test split of 0.35: 0.47
test accuracy with test split of 0.40: 0.49
test accuracy with test split of 0.45: 0.50
test accuracy with test split of 0.50: 0.55


In [ ]:
#Now, use the parameters of the best model found in 4.6

#reg  num_iters  step_size  accuracy_mean  accuracy_std
#0.52        200        1.0         0.6325      0.065383


In [64]:
#Initial value for the Gradient Descent Parameter
step_size = 1.0 #learning rate

num_iterations = 200 # iterations

#Initialize hyper-parameter value (optimized in 4.6)
reg = 0.52 # regularization strength

#iterate over traning grid
for index, row in trainGrid_diffSplits.iterrows():
    #random weights
    W = 0.01 * np.random.randn(D,K)
    #zero bias terms
    b = np.zeros((1,K))

    trainX, testX, trainY, testY = train_test_split(X,y, test_size = row['test_size'], stratify = y)
    
    num_examples = trainX.shape[0]
    
    # gradient descent loop
    for i in range(num_iterations): 

        # evaluate class scores, [N x K]
        # X = 300 obs * 2 features || W = 2 (features) * 3 (output classes) || b = bias weights
        scores = np.dot(trainX, W) + b 

        # compute the class probabilities
        exp_scores = np.exp(scores)

        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True) # [N x K]

        #Compute loss using snippet of LogLoss fucntion created in Q3
        #convert the y label to a N * K array 
        actual = np.zeros(probs.shape)
        rows = actual.shape[0]

        actual[np.arange(rows), trainY.astype(int)] = 1

        #compute data loss
        data_loss = -1.0 / rows * (np.sum(actual * np.log(probs)))

        # L2 regularization term
        L2 = np.sum(W ** 2) #not multiplied by 0.5

        # the total loss
        loss = data_loss + reg * L2

        #if i % 10 == 0:
            #print "iteration %d: loss %f" % (i, loss)
            #print "iteration %d: loss %f" % (i, loss_old)

        # compute the gradient on scores
        dscores = probs
        dscores[range(num_examples),trainY] -= 1
        dscores /= num_examples

        # backpropate the gradient to the parameters (W,b)
        dW = np.dot(trainX.T, dscores)
        db = np.sum(dscores, axis=0, keepdims=True)

        dW += reg*W # regularization gradient

        # perform a parameter update
        W += -step_size * dW
        b += -step_size * db

    scores = np.dot(testX, W) + b
    predicted_class = np.argmax(scores, axis=1)
    trainGrid_diffSplits.set_value(index,'accuracy', np.mean(predicted_class == testY))
    print ('test accuracy with test split of %.2f: %.2f' % (row['test_size'], np.mean(predicted_class == testY)))
    

test accuracy with test split of 0.05: 0.73
test accuracy with test split of 0.10: 0.60
test accuracy with test split of 0.15: 0.65
test accuracy with test split of 0.20: 0.55
test accuracy with test split of 0.25: 0.43
test accuracy with test split of 0.30: 0.47
test accuracy with test split of 0.35: 0.46
test accuracy with test split of 0.40: 0.50
test accuracy with test split of 0.45: 0.51
test accuracy with test split of 0.50: 0.55
